<h1>What factors influence Salary the most? Education or Projections of Jobs in the specific field? </h1>

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
url = "https://money.usnews.com/careers/best-jobs/rankings/the-100-best-jobs/"
#headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
#html = requests.get(url, headers=headers)
#But couldn't simply use request to get the data because the site is dynamic and needed scrolling and clicking to get more data

<h1>Use Selenium to scroll down the page and get all jobs</h1>

<b> Site probably detected automation was used, so hidden data would not appear after scrolling.</b>
<p>I headed to the site below to figure out how to overcome that:</p>
<a href="https://piprogramming.org/articles/How-to-make-Selenium-undetectable-and-stealth--7-Ways-to-hide-your-Bot-Automation-from-Detection-0000000017.html"> 12 Ways to hide your bot automation </a>

In [6]:
option = webdriver.ChromeOptions()

#Removes navigator.webdriver flag
option.add_argument('--disable-blink-features=AutomationControlled')

#Open Browser
driver = webdriver.Chrome(executable_path='chromedriver.exe',options=option)
driver.maximize_window()
driver.get(url)


#Now to the actual Scrolling
pause_time = 10 #Seconds

#Get the length of the page
first_height = driver.execute_script("return document.body.scrollHeight") #"body" can be replaced with "documentElement"
print("Initial Page Length: ", first_height)

def load():
    """Function To Scroll Down"""
    global driver, first_height
    
    #Actual Scrolling command
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(pause_time) #Give time to load the next chunck
    
    # Calculate new scroll height and compare with last scroll height
    current_height = driver.execute_script("return document.body.scrollHeight")
    print("Current Page Length: ", current_height)
    if current_height == first_height:
        return 0;
    first_height = current_height

#Load Scroll many times
while True:
    load()
        
    if load() == 0: #Load returns 0 when the page height remained unchanged after a new scroll
        print("Hit the bottom!")
        time.sleep(pause_time) #Give time to load next chunk if there is still more
        
        #Now click on the load button to load the remaining data
        try:
            driver.find_elements_by_xpath("//*[text()='Load More']")[0].click() #Click on the Load Button
            print("Load More Button Clicked!\nLoading More Data...")
            load() #Load the next chuncks
        except:
            print("Reached bottom of page, no further data to load.")
            break

Initial Page Length:  4759
Current Page Length:  14765
Current Page Length:  17989
Current Page Length:  17989
Current Page Length:  17989
Hit the bottom!
Load More Button Clicked!
Loading More Data...
Current Page Length:  21237
Current Page Length:  24719
Current Page Length:  27919
Current Page Length:  31119
Current Page Length:  34625
Current Page Length:  34625
Current Page Length:  34625
Hit the bottom!
Reached bottom of page, no further data to load.


In [7]:
soup = BeautifulSoup(driver.page_source, "html.parser")

<h1> Create a dataframe with only column names </h1
<h2> and then add rows with df.loc[index] = [list of values] for each data entry.</h2>

In [8]:
df = pd.DataFrame(columns=["Title", "Rank", "Projected Jobs", "Median Salary", "Education Needed"])

In [9]:
#All the jobs on the page are contained in a div tag with class = "card-content-container"
jobs = soup.find_all("div", class_="card-content-container")

In [10]:
index = 0

for job in jobs:
    title = job.find("h2").string
    rank = job.span.span.text.lstrip("#")

    data = job.find_all("dd")
    projected_jobs = data[0].a.string;   """Remove comma -->"""; projected_jobs = "".join(projected_jobs.split(","))
    median_salary = data[1].a.string.lstrip("$"); """Remove comma -->"""; median_salary = "".join(median_salary.split(","))
    try:
        education_needed = data[2].a.string
    except Exception:
        education_needed = "Not required"
    
    df.loc[index] = [title, rank, projected_jobs, median_salary, education_needed]
    
    index += 1

In [11]:
df.head(10)

,Title,Rank,Projected Jobs,Median Salary,Education Needed
0,Information Security Analyst,1,47100,103590,Bachelor's
1,Nurse Practitioner,2,114900,111680,Master's
2,Physician Assistant,3,40100,115390,Master's
3,Medical and Health Services Manager,4,139600,104280,Bachelor's
4,Software Developer,5,409500,110140,Bachelor's
5,Data Scientist,6,19800,98230,Bachelor's
6,Financial Manager,7,118200,134180,Bachelor's
7,Statistician,8,14900,92270,Master's
8,Lawyer,9,71500,126930,Doctorate
9,Speech-Language Pathologist,10,45400,80480,Master's


In [70]:
df[["Projected Jobs", "Rank", "Median Salary"]] = df[["Projected Jobs", "Rank", "Median Salary"]].astype(int)
df.dtypes

Title               object
Rank                 int32
Projected Jobs       int32
Median Salary        int32
Education Needed    object
dtype: object

In [22]:
df.to_csv("best_jobs_scraped.csv", index=False)

In [23]:
df.corr()

,Rank,Projected Jobs,Median Salary
Rank,1.000000,-0.098079,-0.401447
Projected Jobs,-0.098079,1.000000,-0.292828
Median Salary,-0.401447,-0.292828,1.000000


In [43]:
%matplotlib inline

<b>Median Salary is the target variable</b>
<p>Trying to see the relationship between the target and the other variables.</p>

In [90]:
sns.regplot(x='Projected Jobs', y='Median Salary', data=df)
plt.ylim(0,)

(0.0, 225266.65840862633)

In [91]:
sns.regplot(x='Rank', y='Median Salary', data=df)

<AxesSubplot:xlabel='Rank', ylabel='Median Salary'>

Both have very weak linear relationships

In [97]:
#For categorical variables
#plt.figure(figsize=(40,30))  #But this Changes all figures size settings
sns.set(rc={'figure.figsize':(25, 8), 'figure.dpi':600, 'savefig.dpi':600}, font_scale=1.5, style='whitegrid')
sns.boxplot(x="Education Needed", y="Median Salary", data=df)

<AxesSubplot:xlabel='Education Needed', ylabel='Median Salary'>

Education seems to be a very good indicator

<b>Basic Descriptive Stats</b>

In [52]:
df.describe()

,Rank,Projected Jobs,Median Salary
count,100.000000,1.000000e+02,100.000000
mean,50.470000,7.405600e+04,87453.000000
std,29.023312,1.727143e+05,45534.558094
min,1.000000,-7.000000e+03,24960.000000
25%,25.750000,5.975000e+03,55717.500000
50%,50.500000,2.600000e+04,77145.000000
75%,75.250000,6.407500e+04,106870.000000
max,100.000000,1.129900e+06,208000.000000


<b>Basic Grouping</b>

In [60]:
df['Education Needed'].unique()

array(["Bachelor's", "Master's", 'Doctorate', "Associate's",
       'High School', 'Postsecondary non-degree', 'Not required',
       'Some college, no degree'], dtype=object)

In [65]:
educations_and_salary = df[["Education Needed", "Median Salary"]]
salary_by_education = educations_and_salary.groupby(["Education Needed"], as_index = False).mean()
salary_by_education

,Education Needed,Median Salary
0,Associate's,69123.750000
1,Bachelor's,90106.578947
2,Doctorate,148724.375000
3,High School,42270.000000
4,Master's,96366.666667
5,Not required,29812.000000
6,Postsecondary non-degree,45826.666667
7,"Some college, no degree",48225.000000


In [71]:
educations_and_jobs = df[["Education Needed", "Projected Jobs"]]
jobs_by_education = educations_and_jobs.groupby(["Education Needed"], as_index = False).mean()
jobs_by_education

,Education Needed,Projected Jobs
0,Associate's,20475.000000
1,Bachelor's,65131.578947
2,Doctorate,11487.500000
3,High School,274980.000000
4,Master's,20453.333333
5,Not required,233700.000000
6,Postsecondary non-degree,47200.000000
7,"Some college, no degree",37350.000000


In [96]:
plt.scatter(df["Projected Jobs"], df["Median Salary"])
plt.xlabel("Projected Jobs")
plt.ylabel("Median Salary")
plt.show()

In [72]:
from sklearn.linear_model import LinearRegression

In [73]:
lm = LinearRegression()
x = df[["Projected Jobs"]]
y = df[["Median Salary"]]
lm.fit(x, y)

LinearRegression()

In [78]:
lm.predict([[12000], [47100]])

array([[92243.80847208],
       [89534.04024064]])